In [6]:
import json
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

#ML
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn import svm
from sklearn.svm import SVC
from scipy.sparse import coo_matrix, hstack

#NLP
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
import pandas as pd
import numpy as np


# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *


In [7]:
#1. Train Data
with open('../data/train.json') as fin:
    trainjson = json.load(fin)
train = pd.io.json.json_normalize(trainjson)
#2. Test Data
with open('../data/test.json') as fin:
    testjson = json.load(fin)
test = pd.io.json.json_normalize(testjson)

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

Train Shape: (4040, 32)
Test Shape: (1631, 17)


In [18]:

pd_train = train
pd_test = test


np_test = np.array(pd_test)
np_train = np.array(pd_train)

print(np_train.shape)


X = np_train[:,6]
Y = np_train[:,22]
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]


print('data shape: ', X.shape)
print('label shape:', Y.shape)



train_data, test_data, train_labels, test_labels = train_test_split(
 X,
 Y, test_size=0.2, random_state=0)

test_data, dev_data, test_labels, dev_labels = train_test_split(
 test_data,
 test_labels, test_size=0.2, random_state=0)

(4040, 32)
data shape:  (4040,)
label shape: (4040,)


In [20]:

vect=CountVectorizer()
data=vect.fit_transform(train_data).toarray()
devdata=vect.transform(dev_data).toarray()


#Use np.where to binarize train and dev set where values above and below 0.5.
b=train_labels
trainlabels=np.where(b==True, 1, 0)

bl=dev_labels
devlabels=np.where(bl==True, 1, 0)

b2=test_labels
testlabels=np.where(b2==True, 1, 0)

categories = ['Got Pizza', 'Didn\'t get pizza']

print('Baseline Scores...')
#Run MultinomialNB Classifier
# mnb_clf = Pipeline([('vect', CountVectorizer()), ('mnclf',MultinomialNB(alpha=0.01))])
# mnb_clf = mnb_clf.fit(train_data, trainlabels)
# pred = mnb_clf.predict(dev_data)
# score1=metrics.accuracy_score(devlabels,pred)
# print 'Naive Bayes Score:',score1
best_nb = []
alphas = [0.0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]
for k in range(len(alphas)):
    mnb_clf = Pipeline([('vect', CountVectorizer()), ('mnclf', MultinomialNB(alpha=alphas[k]))])
    mnb_clf = mnb_clf.fit(train_data, trainlabels)
    pred = mnb_clf.predict(dev_data)
    metrics.accuracy_score(devlabels,pred)
    best_nb.append(metrics.accuracy_score(devlabels,pred))
bestAlphaAccuracy = max(best_nb)
bestAlphaValue = alphas[best_nb.index(bestAlphaAccuracy)]
print('Naive Bayes Baseline:')
print('Best Alpha =', bestAlphaValue, ' accuracy:', bestAlphaAccuracy)
print('')



#Run Logistic Regression classifier
log_clf = Pipeline([('vect', CountVectorizer()),('lgclf', LogisticRegression(C=0.5))])
log_clf = log_clf.fit(train_data, trainlabels) 
pred = log_clf.predict(dev_data)        
score2= metrics.accuracy_score(devlabels,pred)
#print 'Logistic Regression Score:',score2
best_logit = []
C = [0.0001, 0.001, 0.01, 0.1, 0.5, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
for k in range(len(C)):
    log_clf = Pipeline([('vect', CountVectorizer()),
                     ('lgclf', LogisticRegression(C=C[k]))]);
    log_clf = log_clf.fit(train_data, trainlabels)
    pred = log_clf.predict(dev_data)
    metrics.accuracy_score(devlabels,pred)
    best_logit.append(metrics.accuracy_score(devlabels,pred))
    weights = log_clf.named_steps['lgclf'].coef_
bestCAccuracy = max(best_logit)
bestCValue = C[best_logit.index(bestCAccuracy)]
print('Logistic Regression Baseline:')
print('Best C =', bestCValue, ' accuracy:', bestCAccuracy)
print('')



Baseline Scores...


C:\Users\zmerritt\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


Naive Bayes Baseline:
Best Alpha = 0.5  accuracy: 0.734567901235

Logistic Regression Baseline:
Best C = 0.1  accuracy: 0.740740740741

